Perceptron algorithm

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def main():
  print('****************************************************************************************')
  print('The classifier accuracy for class-1 & class-2 on training dataset:\n',test('train.data',1,2))
  print('The classifier accuracy for class-1 & class-2 on testing dataset:\n',test('test.data',1,2))
  print('The classifier accuracy for class-2 & class-3 on training dataset:\n',test('train.data',2,3))
  print('The classifier accuracy for class-2 & class-3 on testing dataset:\n',test('test.data',2,3))
  print('The classifier accuracy for class-1 & class-3 on training dataset:\n',test('train.data',1,3))
  print('The classifier accuracy for class-1 & class-3 on testing dataset:\n',test('test.data',1,3))
  print('****************************************************************************************')
  print('The accuracy of multi-class classification on training dataset:\n',one_vs_rest('train.data'))
  print('The accuracy of multi-class classification on testing dataset:\n',one_vs_rest('test.data'))
  print('****************************************************************************************')
  print('The accuracy of multi-class classification with l2 regularisation on training dataset:')
  print('Accurcy with regularisation coefficient 0.01 on training data:',l2_onevsrest('train.data',0.01))
  print('Accurcy with regularisation coefficient 0.01 on testing data:',l2_onevsrest('test.data',0.01))
  print('Accurcy with regularisation coefficient 0.1 on training data:',l2_onevsrest('train.data',0.1))
  print('Accurcy with regularisation coefficient 0.1 on testing data:',l2_onevsrest('test.data',0.1))
  print('Accurcy with regularisation coefficient 1.0 on training data:',l2_onevsrest('train.data',1.0))
  print('Accurcy with regularisation coefficient 1.0 on testing data:',l2_onevsrest('test.data',1.0))
  print('Accurcy with regularisation coefficient 10.0 on training data:',l2_onevsrest('train.data',10.0))
  print('Accurcy with regularisation coefficient 10.0 on testing data:',l2_onevsrest('test.data',10.0))
  print('Accurcy with regularisation coefficient 100.0 on training data:',l2_onevsrest('train.data',100.0))
  print('Accurcy with regularisation coefficient 100.0 on testing data:',l2_onevsrest('test.data',100.0))
  print('****************************************************************************************')

In [ ]:
main()

In [ ]:
def load_data(filename):
  data = pd.read_csv(filename, header = None)
  #data = data[:]
# replace the string data class name with number
  data = data.replace(['class-1'],1)
  data = data.replace(['class-2'],2)
  data = data.replace(['class-3'],3)
# randomise the dataframe by row and keep the same order in each execution
  data = data.sample(frac=1, random_state=42)
# output the data as matrix
  data = np.asmatrix(data)
  return data

In [ ]:
# fuction form a new dataset contains spcific classes
# use a, b parameteres to choose the classes in the new dataset
def separate_data(data, a, b):  
  numRecords = len(data[:, :])
  new_data = np.empty((0,5))
  for index in range(numRecords):
    if data[index, -1] == a or data[index, -1] == b:
# in the new dataset set class-a as 1, class-b as -1
      data[index,-1] = np.where(data[index,-1] == a, 1, -1)
      new_data = np.vstack((new_data, data[index]))
  return new_data

In [ ]:
# fuction form a new dataset that class-a as positive, other classes as negetive
def multiclass(a):
  data = load_data('train.data')
  numRecords = len(data)
  new_data = np.empty((0,5))
  for index in range(numRecords):
    data[index,-1] = np.where(data[index,-1] == a, 1, -1)
    new_data = np.vstack((new_data, data[index]))
  return new_data

In [ ]:
def plot(a, b):
  data = load_data('train.data')
  data = separate_data(data, a, b)
  plt.scatter(np.array(data[:40,0]), np.array(data[:40,2]), marker='o', label='class-1')
  plt.scatter(np.array(data[40:,0]), np.array(data[40:,2]), marker='x', label='class-2')
  plt.xlabel('petal length')
  plt.ylabel('sepal length')
  plt.legend()
  plt.show()
plot(2,3)

In [ ]:
def perceptron(train_data, num_iter):
# split actual data and class label in the dataset
  features = train_data[:, :-1]
  labels = train_data[:, -1]
# set weights and bias to zero 
  w = np.zeros((1, features.shape[1]+1))
# number of iteration 
  for iteration in range(num_iter):
    for x, label in zip(features, labels):
# put a 1 at the begining of each line for bias in w
      x = np.insert(x,0,1)
# the activation score WTX
      activ = np.inner(w, x)
# if the label and activation score has different sign
      if (label * activ) <= 0:
#perform the update rule
        w += label * x
  return w

In [ ]:
# perceptron using l2 update rule
def l2_perceptron(train_data, num_iter, lambda_):
  features = train_data[:, :-1]
  labels = train_data[:, -1]
  w = np.zeros((1, features.shape[1]+1))
  for iteration in range(num_iter):
    for x, label in zip(features, labels):
      x = np.insert(x,0,1)
      activ = np.inner(w, x)
      if (label * activ) <= 0:
        w = (1 - 2 * lambda_) * w + label * x
  return w

In [ ]:
# output the accuracy of the binary perception between class-a & class-b
def test(testfile, a, b):
  data = load_data('train.data')
  data = separate_data(data, a, b)
  w = perceptron(data, 20)
  correctPredictions = 0
  #validation_data = load_data(testfile)
  #validation_data = separate_data(validation_data, a, b)
  features = data[:, :-1]
  labels = data[:, -1]
  for x, label in zip(features, labels):
    x = np.insert(x,0,1)
    activ = np.inner(w, x)
    if activ * label > 0:
      correctPredictions += 1
  accuracy = float(correctPredictions) / float(len(data))
  return accuracy
#print("Accuracy of perceptron: ", test('test.data',1,2))   
#print("Accuracy of perceptron: ", test('train.data',2,3))  

In [ ]:
# multi-class classification using one-vs-rest then output the accuracy
def one_vs_rest(filename):
# form three new dataset which set the corresponding class to positive, others to negetive
  data_1 = multiclass(1)
  data_2 = multiclass(2)
  data_3 = multiclass(3)
# use perceptron algorithm to get the weight vector after 20 iterations for each set
  w1 = perceptron(data_1, 20)
  w2 = perceptron(data_2, 20)
  w3 = perceptron(data_3, 20)
  correctPredictions = 0
# load the validation data
  validation_data = load_data(filename)
  features = validation_data[:, :-1]
  labels = validation_data[:, -1]
  for x, label in zip(features, labels):
    x = np.insert(x,0,1)
# three activation scores calculated using result from 3 prediction models
    activ = np.empty((3))
    activ[0] = np.inner(w1,x)
    activ[1] = np.inner(w2,x)
    activ[2]= np.inner(w3,x)
# find the max value among three activation scores
    activ = np.argmax(activ)
# assign corresponding class label to the prediction
    if activ == 0:
      activ = 1
    elif activ == 1:
      activ = 2
    else:
      activ = 3
# verify the prediction
    if activ == label:
      correctPredictions += 1
# calculate the accuracy
  accuracy = float(correctPredictions) / float(len(validation_data))
  return accuracy

In [ ]:
# multi-class classifier with l2
def l2_onevsrest(filename, lambda_):
  data_1 = multiclass(1)
  data_2 = multiclass(2)
  data_3 = multiclass(3)
  w1 = l2_perceptron(data_1, 20, lambda_)
  w2 = l2_perceptron(data_2, 20, lambda_)
  w3 = l2_perceptron(data_3, 20, lambda_)
  correctPredictions = 0
  validation_data = load_data(filename)
  features = validation_data[:, :-1]
  labels = validation_data[:, -1]
  for x, label in zip(features, labels):
    x = np.insert(x,0,1)
    activ = np.empty((3))
    activ[0] = np.inner(w1,x)
    activ[1] = np.inner(w2,x)
    activ[2] = np.inner(w3,x)
    activ = np.argmax(activ)
    if activ == 0:
      activ = 1
    elif activ == 1:
      activ = 2
    else:
      activ = 3
    if activ == label:
      correctPredictions += 1
  accuracy = float(correctPredictions) / float(len(validation_data))
  return accuracy

In [ ]:
"""COMP527 Data Mining and Visualisation
Coursework 1-Perceptron algorithm
Jining Liu-201476244
"""

import numpy as np
import pandas as pd


def load_data(filename):
    data = pd.read_csv(filename, header=None)
    # data = data[:]
    # replace the string data class name with number
    data = data.replace(['class-1'], 1)
    data = data.replace(['class-2'], 2)
    data = data.replace(['class-3'], 3)
    # randomise the dataframe by row and keep the same order in each execution
    data = data.sample(frac=1, random_state=20)
    # output the data as matrix
    data = np.asmatrix(data)
    return data


# fuction form a new dataset contains spcific classes
# use a, b parameteres to choose the classes in the new dataset
def separate_data(data, a, b):
    numRecords = len(data[:, :])
    new_data = np.empty((0, 5))
    for index in range(numRecords):
        if data[index, -1] == a or data[index, -1] == b:
            # in the new dataset set class-a as 1, class-b as -1
            data[index, -1] = np.where(data[index, -1] == a, 1, -1)
            new_data = np.vstack((new_data, data[index]))
    return new_data


# fuction form a new dataset that class-a as positive, other classes as negetive
def multiclass(a):
    data = load_data('train.data')
    numRecords = len(data)
    new_data = np.empty((0, 5))
    for index in range(numRecords):
        data[index, -1] = np.where(data[index, -1] == a, 1, -1)
        new_data = np.vstack((new_data, data[index]))
    return new_data


def perceptron(train_data, num_iter):
    # split actual data and class label in the dataset
    features = train_data[:, :-1]
    labels = train_data[:, -1]
    # set weights and bias to zero
    w = np.zeros((1, features.shape[1] + 1))
    # number of iteration
    for iteration in range(num_iter):
        for x, label in zip(features, labels):
            # put a 1 at the begining of each line for bias in w
            x = np.insert(x, 0, 1)
            # the activation score WTX
            activ = np.inner(w, x)
            # if the label and activation score has different sign
            if (label * activ) <= 0:
                # perform the update rule
                w += label * x
    return w


# perceptron using l2 update rule
def l2_perceptron(train_data, num_iter, lambda_):
    features = train_data[:, :-1]
    labels = train_data[:, -1]
    w = np.zeros((1, features.shape[1] + 1))
    for iteration in range(num_iter):
        for x, label in zip(features, labels):
            x = np.insert(x, 0, 1)
            activ = np.inner(w, x)
            if (label * activ) <= 0:
                w = (1 - 2 * lambda_) * w + label * x
    return w


# output the accuracy of the binary perception between class-a & class-b
def test(testfile, a, b):
    data = load_data('train.data')
    data = separate_data(data, a, b)
    w = perceptron(data, 20)
    correctPredictions = 0
    # validation_data = load_data(testfile)
    # validation_data = separate_data(validation_data, a, b)
    features = data[:, :-1]
    labels = data[:, -1]
    for x, label in zip(features, labels):
        x = np.insert(x, 0, 1)
        activ = np.inner(w, x)
        if activ * label > 0:
            correctPredictions += 1
    accuracy = float(correctPredictions) / float(len(data))
    return accuracy


# print("Accuracy of perceptron: ", test('test.data',1,2))
# print("Accuracy of perceptron: ", test('train.data',2,3))

# multi-class classification using one-vs-rest then output the accuracy
def one_vs_rest(filename):
    # form three new dataset which set the corresponding class to positive, others to negetive
    data_1 = multiclass(1)
    data_2 = multiclass(2)
    data_3 = multiclass(3)
    # use perceptron algorithm to get the weight vector after 20 iterations for each set
    w1 = perceptron(data_1, 20)
    w2 = perceptron(data_2, 20)
    w3 = perceptron(data_3, 20)
    correctPredictions = 0
    # load the validation data
    validation_data = load_data(filename)
    features = validation_data[:, :-1]
    labels = validation_data[:, -1]
    for x, label in zip(features, labels):
        x = np.insert(x, 0, 1)
        # three activation scores calculated using result from 3 prediction models
        activ = np.empty((3))
        activ[0] = np.inner(w1, x)
        activ[1] = np.inner(w2, x)
        activ[2] = np.inner(w3, x)
        # find the max value among three activation scores
        activ = np.argmax(activ)
        # assign corresponding class label to the prediction
        if activ == 0:
            activ = 1
        elif activ == 1:
            activ = 2
        else:
            activ = 3
        # verify the prediction
        if activ == label:
            correctPredictions += 1
    # calculate the accuracy
    accuracy = float(correctPredictions) / float(len(validation_data))
    return accuracy


# multi-class classifier with l2
def l2_onevsrest(filename, lambda_):
    data_1 = multiclass(1)
    data_2 = multiclass(2)
    data_3 = multiclass(3)
    w1 = l2_perceptron(data_1, 20, lambda_)
    w2 = l2_perceptron(data_2, 20, lambda_)
    w3 = l2_perceptron(data_3, 20, lambda_)
    correctPredictions = 0
    validation_data = load_data(filename)
    features = validation_data[:, :-1]
    labels = validation_data[:, -1]
    for x, label in zip(features, labels):
        x = np.insert(x, 0, 1)
        activ = np.empty((3))
        activ[0] = np.inner(w1, x)
        activ[1] = np.inner(w2, x)
        activ[2] = np.inner(w3, x)
        activ = np.argmax(activ)
        if activ == 0:
            activ = 1
        elif activ == 1:
            activ = 2
        else:
            activ = 3
        if activ == label:
            correctPredictions += 1
    accuracy = float(correctPredictions) / float(len(validation_data))
    return accuracy


def main():
    print('****************************************************************************************')
    print('The classifier accuracy for class-1 & class-2 on training dataset:\n', test('train.data', 1, 2))
    print('The classifier accuracy for class-1 & class-2 on testing dataset:\n', test('test.data', 1, 2))
    print('The classifier accuracy for class-2 & class-3 on training dataset:\n', test('train.data', 2, 3))
    print('The classifier accuracy for class-2 & class-3 on testing dataset:\n', test('test.data', 2, 3))
    print('The classifier accuracy for class-1 & class-3 on training dataset:\n', test('train.data', 1, 3))
    print('The classifier accuracy for class-1 & class-3 on testing dataset:\n', test('test.data', 1, 3))
    print('****************************************************************************************')
    print('The accuracy of multi-class classification on training dataset:\n', one_vs_rest('train.data'))
    print('The accuracy of multi-class classification on testing dataset:\n', one_vs_rest('test.data'))
    print('****************************************************************************************')
    print('The accuracy of multi-class classification with l2 regularisation on training dataset:')
    print('Accurcy with regularisation coefficient 0.01 on training data:', l2_onevsrest('train.data', 0.01))
    print('Accurcy with regularisation coefficient 0.01 on testing data:', l2_onevsrest('test.data', 0.01))
    print('Accurcy with regularisation coefficient 0.1 on training data:', l2_onevsrest('train.data', 0.1))
    print('Accurcy with regularisation coefficient 0.1 on testing data:', l2_onevsrest('test.data', 0.1))
    print('Accurcy with regularisation coefficient 1.0 on training data:', l2_onevsrest('train.data', 1.0))
    print('Accurcy with regularisation coefficient 1.0 on testing data:', l2_onevsrest('test.data', 1.0))
    print('Accurcy with regularisation coefficient 10.0 on training data:', l2_onevsrest('train.data', 10.0))
    print('Accurcy with regularisation coefficient 10.0 on testing data:', l2_onevsrest('test.data', 10.0))
    print('Accurcy with regularisation coefficient 100.0 on training data:', l2_onevsrest('train.data', 100.0))
    print('Accurcy with regularisation coefficient 100.0 on testing data:', l2_onevsrest('test.data', 100.0))
    print('****************************************************************************************')


main()

In [ ]:
import numpy as np

def nonlin(x, deriv = False):
    if(deriv == True):
        return x * (1 - x)
    return 1 / (1 + np.exp(-x))


X = np.array([[0.35], [0.9]])
y = np.array([[0.5]])

np.random.seed(1)

W0 = np.array([[0.1, 0.8], [0.4, 0.6]])
W1 = np.array([[0.3, 0.9]])

print ('original ', W0, '\n', W1)

for j in range(100):
    l0 = X
    l1 = nonlin(np.dot(W0, l0))
    l2 = nonlin(np.dot(W1, l1))
    l2_error = y - l2
    Error = 1 / 2.0 * (y-l2)**2
    print ('Error:', Error)

    l2_delta = l2_error * nonlin(l2, deriv=True)

    l1_error = l2_delta * W1 #back propagation
    l1_delta = l1_error * nonlin(l1, deriv=True)

    W1 += l2_delta * l1.T
    W0 += l0.T.dot(l1_delta)
    print (W0, '\n', W1)